# Deep Research tool with Azure AI Foundry (preview)

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

The **Deep Research tool** in the Azure AI Foundry Agent Service enables you to integrate a web-based research capability into your systems. The Deep Research capability is a specialized AI capability designed to perform in-depth, multi-step research using data from the public web.

> https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/deep-research

The **o3-deep-research model** and the GPT model deployments should be part of your AI Foundry project resulting in all three resources in the same Azure subscription and same region. Supported regions are **West US and Norway East.**

At its core, **Deep Research** leverages a combination of OpenAI and Microsoft technologies, including **o3-deep-research**, various GPT models, and **Bing Search Grounding**, when integrated into an agent.

- When an agent with Deep Research integration receives a research request — whether from a user or another application — it utilizes GPT-4o and GPT-4.1 to interpret the intent, fill in any missing details, and define a clear, actionable scope for the task.
- Once the task is defined, the agent activates the Bing-powered grounding tool to gather a refined selection of recent, high-quality web content.
- Following this, the o3-deep-research agent begins the research process by reasoning through the collected information. Rather than merely summarizing content, it evaluates, adapts, and synthesizes insights from multiple sources, adjusting its approach as new data emerges.
- The entire process culminates in a structured report that not only provides the answer but also includes the model’s reasoning path, source citations, and any clarifications requested during the session, as explained by Arenas.

In [ ]:
#%pip install --pre azure-ai-projects
#%pip install pypandoc
#%pip install azure-identity

In [ ]:
import datetime
import os
import pypandoc
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import DeepResearchTool, MessageRole, ThreadMessage
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from IPython.display import display, FileLink, Markdown
from typing import Optional

In [ ]:
sys.version

In [ ]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

## Settings

> Only available now in "West US" and "Norway East"

In [ ]:
# Foundry project
project_endpoint = "https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject"

In [ ]:
model = "gpt-4.1"  # a generic gpt model
deep_research_model = "o3-deep-research"  # the new o3 deep research model deployed in your AI Foundry

bingservice = "aqbinggrounding002"  # The Bing connection service in your AI foundry project

In [ ]:
RESULTS_DIR = "documents"

os.makedirs(RESULTS_DIR, exist_ok=True)

In [ ]:
now = datetime.datetime.today().strftime('%d%b%Y_%H%M%S')

md_results_file = os.path.join(RESULTS_DIR, f"deep_research_results_{now}.md")  # The name of the markdown output file
docx_file = os.path.join(RESULTS_DIR, f"deep_research_results_{now}.docx")  # .docx outpyt

## Helper

In [ ]:
def fetch_and_print_new_agent_response(
    thread_id: str,
    agents_client: AgentsClient,
    last_message_id: Optional[str] = None,
) -> Optional[str]:
    """
    Fetches and prints the latest response from an agent in a given thread.

    Args:
        thread_id (str): The ID of the thread to fetch the agent's response from.
        agents_client (AgentsClient): The client used to interact with the agents service.
        last_message_id (Optional[str], optional): The ID of the last message that was processed. Defaults to None.

    Returns:
        Optional[str]: The ID of the latest message if there is new content, otherwise returns the last_message_id.
    """
    response = agents_client.messages.get_last_message_by_role(
        thread_id=thread_id,
        role=MessageRole.AGENT,
    )
    if not response or response.id == last_message_id:
        return last_message_id  # No new content

    print("\nAgent response:")
    print("\n".join(t.text.value for t in response.text_messages))

    for ann in response.url_citation_annotations:
        print(
            f"URL Citation: [{ann.url_citation.title}]({ann.url_citation.url})"
        )

    return response.id

In [ ]:
def create_research_summary(message: ThreadMessage,
                            filepath: str = md_results_file) -> None:
    """
    Creates a research summary from the provided message and writes it to a file.

    Args:
        message (ThreadMessage): The message containing the content for the research summary.
        filepath (str, optional): The path to the file where the research summary will be written in a markdown format.

    Returns:
        None
    """
    if not message:
        print("Error: No message content provided")
        return

    with open(filepath, "w", encoding="utf-8") as fp:
        # Write text summary
        text_summary = "\n\n".join(
            [t.text.value.strip() for t in message.text_messages])
        fp.write(text_summary)

        # Write unique URL citations, if present
        if message.url_citation_annotations:
            fp.write("\n\n## References\n")
            seen_urls = set()
            for ann in message.url_citation_annotations:
                url = ann.url_citation.url
                title = ann.url_citation.title or url
                if url not in seen_urls:
                    fp.write(f"- [{title}]({url})\n")
                    seen_urls.add(url)

    print(f"Research summary written to '{filepath}'.")

## Project & tool definitions

In [ ]:
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

conn_id = project_client.connections.get(name=bingservice).id

In [ ]:
deep_research_tool = DeepResearchTool(
    bing_grounding_connection_id=conn_id,
    deep_research_model=deep_research_model,
)

## Example

In [ ]:
prompt = "Give me the latest research into magentic-ui from Microsoft's autogen over the last year. Do not ask questions"

In [ ]:
start = time.time()

with project_client:
    with project_client.agents as agents_client:
        agent = agents_client.create_agent(
            model=model,
            name="deep-research-agent",
            instructions=
            "You are a helpful agent that assists in researching scientific topics.",
            tools=deep_research_tool.definitions,
        )

        print(f"🎉 Created agent, ID: {agent.id}")

        thread = agents_client.threads.create()
        print(f"🧵 Created thread, ID: {thread.id}")

        # Create message to thread
        message = agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=(prompt),
        )
        print(f"📩 Created message, ID: {message.id}")
        print(
            f"⏳ Start processing the message... this may take a few minutes to finish. Be patient!"
        )

        run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)
        last_message_id = None

        while run.status in ("queued", "in_progress"):
            time.sleep(1)
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

            last_message_id = fetch_and_print_new_agent_response(
                thread_id=thread.id,
                agents_client=agents_client,
                last_message_id=last_message_id,
            )
            print(f"🔄 Run status: {run.status}")

        print(f"✅ Run finished with status: {run.status}, ID: {run.id}")

        if run.status == "failed":
            print(f"❌ Run failed: {run.last_error}")

        final_message = agents_client.messages.get_last_message_by_role(
            thread_id=thread.id, role=MessageRole.AGENT)
        if final_message:
            create_research_summary(final_message)

        # Clean-up and delete the agent once the run is finished.
        #agents_client.delete_agent(agent.id)
        #print("🗑️ Deleted agent")

elapsed = time.time() - start
minutes, seconds = divmod(elapsed, 60)
print(f"\nElapsed time = {minutes:.0f} minutes and {seconds:.0f} seconds")

## Results

In [ ]:
!ls -lh $md_results_file

In [ ]:
with open(md_results_file, 'r', encoding='utf-8') as file:
    markdown_content = file.read()
    display(Markdown(markdown_content))

### Exporting results into a docx file

In [ ]:
pypandoc.convert_file(md_results_file, 'docx', outputfile=docx_file)
print(f"{md_results_file} has been converted to {docx_file}\n")

!ls $docx_file -lh

In [ ]:
link = FileLink(path=docx_file)
link